Following https://en.m.wikipedia.org/wiki/Karplus–Strong_string_synthesis


In [1]:
import numpy as np
import wave
import struct
import matplotlib.pyplot as plt
%matplotlib inline
import IPython

In [2]:
sample_rate = 44100

def p(a):
    return IPython.display.Audio( a, rate=sample_rate, autoplay = True )

OK so now we can try and make a simple synth using the delay line and the shift-by-one filter.
Initializing with noise this makes a plucked string type sound.

In [3]:
def kp_standalone():
    freq = 220
    # F = sample / ( N + .5 )
    # so N + .5 = sample / F

    burstlen = int( (sample_rate / freq + 0.5)* 2 )

    samplelen = sample_rate * 3
    result = np.zeros( samplelen )
    noise = np.random.rand( burstlen ) * 2 - 1

    result[0:burstlen] = noise
    delay = noise

    pos = burstlen
    
    filtAtten = 0.4;
    filtWeight = 0.5

    filtAtten = filtAtten / 100  / ( freq / 440 )

    while( pos < samplelen ):
        dpos = pos % burstlen
        dpnext = (pos + 1 ) % burstlen
        dpfill = (pos - 1) % burstlen

        # Simple averaging filter
        filtval = ( filtWeight * delay[ dpos ] + (1.0 - filtWeight ) * delay[ dpnext ] )  * (1.0 - filtAtten)

        result[ pos ] = filtval
        delay[ dpfill ] = filtval
    
        pos = pos + 1

    return result
        
p( kp_standalone() )
# filtAtten


Great! So from that method we want to do a few things

* Make it feel more like a "class" with a step operator
* Change the seed (not just noise; also square saw and chirp)
* Change the filter styles (3 point; comb)
* Handle frequency shifts and stuff while going
* Have non-integral frequencies and interpolate appropriately (do this one last with
an internal buffer we interpolate)

So lets get to work on that!

In [10]:
class KSSynth:
    def __init__(self):
        self.pos = 0
        self.filtWeight = 0.5
        self.filtAtten = 3.0
        
        self.setFreq( 220 )
        
    def setFreq( self, freq ):
        self.freq = freq
        self.burstlen = int( (sample_rate / freq + 0.5 ) * 2 )
        self.filtAttenScaled = self.filtAtten / 100 / ( self.freq / 440 )
        
    def trigger( self ):
        self.resultBuffer = np.random.rand( self.burstlen ) * 2 - 1
        self.delay = self.resultBuffer        
        
    def step( self ):
        dpos = self.pos % self.burstlen
        dpnext = (self.pos + 1 ) % self.burstlen
        dpfill = (self.pos - 1) % self.burstlen

        # Simple averaging filter
        fw = self.filtWeight;
        fa = self.filtAttenScaled;
        filtval = ( fw * self.delay[ dpos ] + (1.0 - fw ) * self.delay[ dpnext ] )  * (1.0 - fa)
        self.delay[ dpfill ] = filtval
    
        self.pos = self.pos + 1      
        return filtval


In [20]:
k = KSSynth()
f = []
for i in range( 13 ):
    fm = pow( 2, (i)/12.0 )
    print( fm )
    k.setFreq( fm * 220 )
    k.trigger()
    res = [ k.step() for i in range( int(sample_rate/2) )]
    f = f + res
    
p( f )


1.0
1.0594630943592953
1.122462048309373
1.189207115002721
1.2599210498948732
1.3348398541700344
1.4142135623730951
1.4983070768766815
1.5874010519681994
1.681792830507429
1.7817974362806785
1.8877486253633868
2.0
